In [10]:
from math import ceil, floor
import pandas as pd

# -------------------
# INPUT PARAMETERS
# -------------------
panel_watt = 650         # W per panel
panel_vmp = 43.2         # V at max power
panel_voc = 49.8         # V open circuit
daily_consumption = 1150 # kWh/day
percent_to_cover = 0.7
sunshine_hours = 5.9

# -------------------
# INVERTER DATA (AC kW from column L)
# -------------------
inverters = [
    {"Model": "VST-40K",  "ACkW": 40,  "MinMPPT": 100, "MaxMPPT": 1000, "TotalStrings": 8},
    {"Model": "VST-50K",  "ACkW": 50,  "MinMPPT": 100, "MaxMPPT": 1000, "TotalStrings": 10},
    {"Model": "VST-60K",  "ACkW": 60,  "MinMPPT": 100, "MaxMPPT": 1000, "TotalStrings": 12},
    {"Model": "VST-100K", "ACkW": 100, "MinMPPT": 150, "MaxMPPT": 1000, "TotalStrings": 18},
    {"Model": "VST-110K", "ACkW": 110, "MinMPPT": 150, "MaxMPPT": 1000, "TotalStrings": 18},
    {"Model": "VST-120K", "ACkW": 120, "MinMPPT": 150, "MaxMPPT": 1000, "TotalStrings": 24},
    {"Model": "VST-125K", "ACkW": 125, "MinMPPT": 150, "MaxMPPT": 1000, "TotalStrings": 24},
    {"Model": "VST-130K", "ACkW": 130, "MinMPPT": 150, "MaxMPPT": 1000, "TotalStrings": 24},
    {"Model": "VST-136K", "ACkW": 136, "MinMPPT": 150, "MaxMPPT": 1000, "TotalStrings": 24},
    {"Model": "VST-140K", "ACkW": 140, "MinMPPT": 150, "MaxMPPT": 1000, "TotalStrings": 24},
    {"Model": "VST-150K", "ACkW": 150, "MinMPPT": 150, "MaxMPPT": 1000, "TotalStrings": 24},
]

# -------------------
# STEP 1: Required solar production (kW)
# -------------------
solar_kw_needed = (daily_consumption * percent_to_cover) / sunshine_hours

# -------------------
# STEP 2: Select inverter based on AC output
# -------------------
for inverter in inverters:
    inverter["InverterCount"] = ceil(solar_kw_needed / inverter["ACkW"])

# Select inverter with minimum required count
best_inverter = min(inverters, key=lambda x: x["InverterCount"])

# -------------------
# STEP 3: String configuration
# -------------------
max_string_size = floor(best_inverter["MaxMPPT"] / panel_voc)
min_string_size = ceil(best_inverter["MinMPPT"] / panel_vmp)

# Estimate string count per inverter based on power required
strings_per_inverter = ceil(
    (solar_kw_needed * 1000) / (best_inverter["InverterCount"] * max_string_size * panel_watt)
)
total_strings = strings_per_inverter * best_inverter["InverterCount"]

# -------------------
# STEP 4: Panel count & PV sizing
# -------------------
panels_needed = max_string_size * total_strings
total_pv_kw = panels_needed * panel_watt / 1000

# -------------------
# OUTPUT
# -------------------
result = {
    "Inverter Selection": best_inverter["Model"],
    "Number of Inverters": best_inverter["InverterCount"],
    "Max String Size": max_string_size,
    "Min String Size": min_string_size,
    "Strings Per Inverter": strings_per_inverter,
    "Total Strings": total_strings,
    "Panels Needed": panels_needed,
    "Total PV kW": total_pv_kw,
    "Solar kW Needed": round(solar_kw_needed, 2)
}

# Output result
df = pd.DataFrame([result])
print(df.to_string(index=False))


Inverter Selection  Number of Inverters  Max String Size  Min String Size  Strings Per Inverter  Total Strings  Panels Needed  Total PV kW  Solar kW Needed
          VST-140K                    1               20                4                    11             11            220        143.0           136.44
